In [1]:
data_var = '2024-01-08'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9243,2024-01-08,Alemanha Bbl,16:00,Tubingen,Wurzburg,2.77,1.45,167.5,1.85,1.95,3.5,2.08,1.71,jBfZYgZK,0.361011,0.689655,0.540541,0.512821,0.050666,0.8,0.2,NaN,NaN,346.186,196.439605,0.567439,148.944,30.006093,0.201459,370.706,281.846,351.50,137.78,1.0,0.0,0.0,0.0,0.442361,0.037216,0.138063,1.23,0.246,7.195122,0.000000,0.0,0.000000,2.07,0.414,1.086957,0.000000,0.0,0.000000
9244,2024-01-08,Argentina Liga A,20:00,Independiente de Oliva,Obras,2.17,1.64,164.5,1.87,1.79,1.5,2.00,1.72,6TXqQ4Fb,0.460829,0.609756,0.534759,0.558659,0.070586,0.8,0.8,NaN,NaN,178.556,69.662280,0.390142,157.156,48.306073,0.307377,149.200,180.390,141.96,194.65,0.0,0.0,0.0,0.0,0.196728,0.030912,0.106446,1.70,0.340,3.441176,0.000000,0.0,0.000000,0.98,0.196,3.265306,0.000000,0.0,0.000000
9245,2024-01-08,Ásia Wasl,14:00,Kuwait SC,Al Muharraq,1.27,3.53,175.5,1.85,1.85,-9.5,2.00,1.72,Sl5Rsgxf,0.787402,0.283286,0.540541,0.540541,0.070688,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,85.85,111.39,0.0,0.0,0.0,0.0,0.665859,0.000000,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9246,2024-01-08,Bulgária Nbl,14:00,CSKA Sofia,Levski,1.12,5.72,157.5,1.85,1.85,-13.5,2.00,1.72,GjHGk7xb,0.892857,0.174825,0.540541,0.540541,0.067682,0.0,0.0,NaN,NaN,122.352,43.267846,0.353634,370.602,367.945039,0.992831,0.000,0.000,98.58,982.50,0.0,1.0,0.0,0.0,0.951079,0.000000,0.106446,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9247,2024-01-08,China Cba,08:35,Beijing,Nanjing Tongxi,1.26,3.69,200.5,1.82,1.92,-9.5,1.99,1.72,jeOQyKLj,0.793651,0.271003,0.549451,0.520833,0.064654,0.4,0.6,NaN,NaN,203.206,92.406602,0.454743,323.880,249.389129,0.770005,146.968,448.410,144.90,737.90,0.0,1.0,0.0,0.0,0.694250,0.037813,0.102921,-0.63,-0.126,-2.063492,0.683137,0.6,-0.083137,-5.00,-1.000,-2.690000,0.332032,0.2,-0.132032
9248,2024-01-08,China Cba,08:35,Jiangsu Dragons,Shandong,2.85,1.40,198.5,1.82,1.92,5.5,2.03,1.67,tMYjrMrN,0.350877,0.714286,0.549451,0.520833,0.065163,0.4,0.6,NaN,NaN,365.574,214.556013,0.586902,215.022,68.460196,0.318387,332.104,338.950,214.83,273.60,0.0,0.0,0.0,0.0,0.482496,0.037813,0.137599,-2.51,-0.502,-3.685259,0.362536,0.3,-0.062536,-1.31,-0.262,-1.526718,0.388937,0.2,-0.188937
9249,2024-01-08,China Cba,08:35,Qingdao,Shenzhen,1.72,2.05,198.5,1.84,1.88,-3.5,2.02,1.70,h02903bi,0.581395,0.487805,0.543478,0.531915,0.069200,0.2,0.6,NaN,NaN,197.012,78.651372,0.399221,166.306,57.377205,0.345010,241.664,182.794,321.48,157.32,0.0,0.0,0.0,0.0,0.123791,0.015207,0.121653,2.73,0.546,1.318681,0.519701,0.9,0.380299,-2.49,-0.498,-2.108434,0.540240,0.4,-0.140240
9250,2024-01-08,China Cba,08:35,Shanghai,Fujian,1.26,3.69,217.5,1.85,1.89,-14.5,2.09,1.64,xhKMx06p,0.793651,0.271003,0.540541,0.529101,0.064654,0.8,0.4,NaN,NaN,182.442,48.439095,0.265504,423.554,251.436295,0.593635,258.706,258.120,148.40,242.97,0.0,0.0,0.0,0.0,0.694250,0.015125,0.170616,0.33,0.066,3.939394,0.573009,0.7,0.126991,-3.34,-0.668,-4.026946,0.303701,0.1,-0.203701
9251,2024-01-08,China Cba,08:35,Tianjin,Shanxi Zhongyu,1.83,1.92,222.5,1.82,1.90,-2.5,2.06,1.66,Wj1DaNDc,0.546448,0.520833,0.549451,0.526316,0.067281,0.6,0.4,NaN,NaN,321.340,117.812046,0.366627,258.678,99.184092,0.383427,233.146,252.254,320.58,202.30,0.0,0.0,0.0,0.0,0.033941,0.030413,0.152066,1.74,0.348,2.385057,0.492827,0.7,0.207173,-3.30,-0.660,-1.393939,0.413547,0.4,-0.013547
9252,2024-01-08,China Cba,08:35,Zhejiang Chouzhou,Guan

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,21:30,Eua Ncaa,Coppin State,Md.-East. Shore,130.5,1.87,0.5083,Over
